In [3]:
import sys
import os

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from core.RobotLink import *
from core.StereoCamera import *
from core.ParticleFilter import *
from core.probability_functions import *
from core.utils import *
import matplotlib.pyplot as plt

import time
import pandas as pd
import numpy as np

In [2]:
psm_arm = RobotLink('journal_dataset/LND.json')
cam = StereoCamera('journal_dataset/camera_calibration.yaml', rectify=True)

In [3]:
# Get a sample data!!
bag = rosbag.Bag('journal_dataset/stationary_camera_2020-06-24-15-49-10.bag')

ecm_pose_list  = []
ecm_pose_ts    = []
ecm_joint_list = []
ecm_joint_ts   = []
psm_joint_list = []
psm_joint_ts   = []
psm_jaw_list   = []
psm_jaw_ts     = []
left_img_list  = []
left_img_ts    = []
right_img_list = []
right_img_ts   = []

for topic, msg, t in bag.read_messages(topics=['/dvrk/ECM/position_cartesian_current', 
                                               '/dvrk/ECM/state_joint_current',
                                               '/dvrk/PSM1/state_joint_current',
                                               '/dvrk/PSM1/state_jaw_current',
                                               '/stereo/left/image',
                                               '/stereo/right/image']):
    if topic == '/dvrk/ECM/position_cartesian_current':
        ecm_pose_list.append(msg)
        ecm_pose_ts.append(msg.header.stamp.to_sec())
    elif topic == '/dvrk/ECM/state_joint_current':
        ecm_joint_list.append(msg)
        ecm_joint_ts.append(msg.header.stamp.to_sec())
    elif topic == '/dvrk/PSM1/state_joint_current':
        psm_joint_list.append(msg)
        psm_joint_ts.append(msg.header.stamp.to_sec())
    elif topic == '/dvrk/PSM1/state_jaw_current':
        psm_jaw_list.append(msg)
        psm_jaw_ts.append(msg.header.stamp.to_sec())
    elif topic == '/stereo/left/image':
        left_img_list.append(msg)
        left_img_ts.append(msg.header.stamp.to_sec())
    elif topic == '/stereo/right/image':
        right_img_list.append(msg)
        right_img_ts.append(msg.header.stamp.to_sec())
    
    # if len(ecm_pose_list) == 1000:
    #     break

bag.close()

ecm_pose_ts  = np.array(ecm_pose_ts)
ecm_joint_ts = np.array(ecm_joint_ts)
psm_joint_ts = np.array(psm_joint_ts)
psm_jaw_ts   = np.array(psm_jaw_ts)
left_img_ts  = np.array(left_img_ts)
right_img_ts = np.array(right_img_ts)

In [4]:
# Load hand-eye transform (this will be hard-coded for now according to the dataset)
f = open('journal_dataset/handeye.yaml')
hand_eye_data = yaml.load(f, Loader=yaml.FullLoader)

bc_T_cam = np.eye(4)
bc_T_cam[:-1, -1] = np.array(hand_eye_data['cam_tvec'])/1000.0
bc_T_cam[:-1, :-1] = axisAngleToRotationMatrix(hand_eye_data['cam_rvec'])

cam_T_b = np.eye(4)
cam_T_b[:-1, -1] = np.array(hand_eye_data['PSM1_tvec'])/1000.0
cam_T_b[:-1, :-1] = axisAngleToRotationMatrix(hand_eye_data['PSM1_rvec'])

In [5]:
# initialize filter
pf = ParticleFilter(num_states=9, 
                    initialDistributionFunc=sampleNormalDistribution,
                    motionModelFunc=additiveGaussianNoise, \
                    #motionModelFunc=lumpedErrorMotionModel,
                    obsModelFunc=pointFeatureObs,
                    num_particles=200)

initalize = True


# output video
left_video  = cv2.VideoWriter("left_video.mp4",  cv2.VideoWriter_fourcc('m', 'p', '4', 'v'), 30, cam.img_size)
right_video = cv2.VideoWriter("right_video.mp4", cv2.VideoWriter_fourcc('m', 'p', '4', 'v'), 30, cam.img_size)


for left_msg_idx, left_img_msg in enumerate(left_img_list):
    # Synchronize the data to the left_img_msg
    ts = left_img_ts[left_msg_idx]

    ecm_pose_msg  = ecm_pose_list[ np.argmin(np.abs(ecm_pose_ts - ts))]
    ecm_joint_msg = ecm_joint_list[np.argmin(np.abs(ecm_joint_ts - ts))]
    psm_joint_msg = psm_joint_list[np.argmin(np.abs(psm_joint_ts - ts))]
    psm_jaw_msg   = psm_jaw_list  [np.argmin(np.abs(psm_jaw_ts - ts))]
    # left_img_msg  = left_img_list[ left_msg_idx]
    right_img_msg = right_img_list[np.argmin(np.abs(right_img_ts - ts))]
    
    try:
        left_img  = np.ndarray(shape=(left_img_msg.height, left_img_msg.width, 3),
                                      dtype=np.uint8, buffer=left_img_msg.data)
        right_img = np.ndarray(shape=(right_img_msg.height, right_img_msg.width, 3),
                                      dtype=np.uint8, buffer=right_img_msg.data)
    except:
        continue

    left_img, right_img = cam.processImage(left_img, right_img)
    
    # Detect segmented keypoints and get new joint angle readings
    detected_keypoints_l, left_img  = segmentColorAndGetKeyPoints(left_img,  draw_contours=True)
    detected_keypoints_r, right_img = segmentColorAndGetKeyPoints(right_img, draw_contours=True)
    new_joint_angles = np.array(psm_joint_msg.position + psm_jaw_msg.position)

    start_t = time.time()
    psm_arm.updateJointAngles(new_joint_angles)
    
    if initalize:
        # Initialize particle filter
        initalize = False
        
        init_kwargs = {
                        "std": np.array([1.0e-3, 1.0e-3, 1.0e-3, # pos
                                         1.0e-2, 1.0e-2, 1.0e-2, # ori
                                         #5.0e-3, 5.0e-3, 0.02
                                         0.0, 0.0, 0.0])   # joints
                      }
        pf.initializeFilter(**init_kwargs)

    else:
        # Predict particle filter
        j_change = new_joint_angles - last_joint_angle_reading

        std_j = np.abs(j_change)*0.01
        std_j[-3:] = 0.0
        
        # pred_kwargs = {
        #                 "std_pos": 2.5e-5, 
        #                 "std_ori": 1.0e-4,
        #                 "robot_arm": psm_arm, 
        #                 "std_j": std_j,
        #                 "nb": 4
        #               }
        
        pred_kwargs = {
                "std_pos": 2.5e-5, 
                "std_ori": 1.0e-4,
                "robot_arm": psm_arm, 
                "std_j": std_j,
                "nb": 4
                }

        pf.predictionStep(**pred_kwargs)
    
    # Update particle filter
    upd_kwargs = {
                    "point_detections": (detected_keypoints_l, detected_keypoints_r), 
                    "robot_arm": psm_arm, 
                    "cam": cam, 
                    "cam_T_b": cam_T_b,
                    "joint_angle_readings": new_joint_angles,
                    "gamma": 0.15
    }
    
    pf.updateStep(**upd_kwargs)
    last_joint_angle_reading = new_joint_angles
    
    correction_estimation = pf.getMeanParticle()
    
    print("Time to update robot: {}ms".format((time.time()-start_t)*1000))
    
    # Project skeleton
    T = poseToMatrix(correction_estimation[:6])  
    new_joint_angles[-(correction_estimation.shape[0]-6):] += correction_estimation[6:]
    psm_arm.updateJointAngles(new_joint_angles)
    
    img_list = projectSkeleton(psm_arm.getSkeletonPoints(), np.dot(cam_T_b, T),
                               [left_img, right_img], cam.projectPoints)
    
    left_video.write( img_list[0])
    right_video.write(img_list[1])
    
left_video.release()
right_video.release()

/home/arclab/anaconda3/envs/tool_tracking/lib/python3.8/site-packages/scipy/stats/_distn_infrastructure.py:1904: RuntimeWarning: invalid value encountered in true_divide
  x = np.asarray((x - loc)/scale, dtype=dtyp)


Time to update robot: 73.76766204833984ms
Time to update robot: 102.03862190246582ms
Time to update robot: 104.39658164978027ms
Time to update robot: 101.97925567626953ms
Time to update robot: 100.22854804992676ms
Time to update robot: 98.00577163696289ms
Time to update robot: 99.07078742980957ms
Time to update robot: 101.52816772460938ms
Time to update robot: 101.32646560668945ms
Time to update robot: 102.33426094055176ms
Time to update robot: 102.98562049865723ms
Time to update robot: 100.41689872741699ms
Time to update robot: 102.14114189147949ms
Time to update robot: 101.06086730957031ms
Time to update robot: 99.45440292358398ms
Time to update robot: 102.2179126739502ms
Time to update robot: 104.08210754394531ms
Time to update robot: 100.64506530761719ms
Time to update robot: 102.2493839263916ms
Time to update robot: 100.07071495056152ms
Time to update robot: 100.799560546875ms
Time to update robot: 101.41515731811523ms
Time to update robot: 101.37295722961426ms
Time to update robo